In [ ]:
import requests
import pandas as pd
import time
import random
from tqdm import tqdm

N_GAMES = 10000       # cantidad de juegos a extraer
OUTPUT_FILE = "steam_games_enriched.csv"
PAUSE_MIN = 0.1
PAUSE_MAX = 0.3

#  Obtener lista completa de juegos
print("Descargando lista de juegos de Steam...")
apps_url = "https://api.steampowered.com/ISteamApps/GetAppList/v2/"
apps_response = requests.get(apps_url).json()
apps = apps_response["applist"]["apps"]
df_apps = pd.DataFrame(apps)
print(f"Total de juegos en Steam: {len(df_apps)}")

# Seleccionar una muestra aleatoria 
df_sample = df_apps.sample(N_GAMES, random_state=42).reset_index(drop=True)
print(f"Muestra seleccionada: {len(df_sample)} juegos\n")

# Función para obtener info detallada de cada juego 
def get_game_info(appid):
    url = f"https://store.steampowered.com/api/appdetails?appids={appid}&cc=us&l=en"
    try:
        r = requests.get(url, timeout=10)
        data = r.json()
        if str(appid) in data and data[str(appid)]["success"]:
            return data[str(appid)]["data"]
    except Exception:
        return None
    return None

# Función para obtener jugadores concurrente
def get_player_count(appid):
    url = f"https://api.steampowered.com/ISteamUserStats/GetNumberOfCurrentPlayers/v1/?appid={appid}"
    try:
        r = requests.get(url, timeout=5)
        data = r.json()
        return data.get("response", {}).get("player_count", None)
    except Exception:
        return None

# Función para obtener información económica in-game
def get_economy_info(appid):
    url = f"https://api.steampowered.com/ISteamEconomy/GetAssetPrices/v1/?appid={appid}"
    try:
        r = requests.get(url, timeout=5)
        data = r.json()
        if "result" in data and data["result"].get("success"):
            assets = data["result"].get("assets", [])
            return True, len(assets)
    except Exception:
        return False, None
    return False, None

# Recolectar información ---
records = []
for appid in tqdm(df_sample["appid"], desc="Extrayendo datos de juegos"):
    info = get_game_info(appid)
    if not info:
        continue

    player_count = get_player_count(appid)
    has_economy, asset_count = get_economy_info(appid)
    
    record = {
        "appid": appid,
        "name": info.get("name"),
        "type": info.get("type"),
        "is_free": info.get("is_free"),
        "price": info.get("price_overview", {}).get("final"),
        "currency": info.get("price_overview", {}).get("currency"),
        "discount_percent": info.get("price_overview", {}).get("discount_percent"),
        "release_date": info.get("release_date", {}).get("date"),
        "metacritic": info.get("metacritic", {}).get("score"),
        "recommendations": info.get("recommendations", {}).get("total"),
        "required_age": info.get("required_age"),
        "dlc_count": len(info.get("dlc", [])) if info.get("dlc") else 0,
        "developers": ",".join(info.get("developers", [])) if info.get("developers") else None,
        "publishers": ",".join(info.get("publishers", [])) if info.get("publishers") else None,
        "categories": ",".join([c["description"] for c in info.get("categories", [])]) if info.get("categories") else None,
        "genres": ",".join([g["description"] for g in info.get("genres", [])]) if info.get("genres") else None,
        "player_count": player_count,
        "has_economy": has_economy,
        "asset_count": asset_count
    }

    records.append(record)
    time.sleep(random.uniform(PAUSE_MIN, PAUSE_MAX))

#  Guardar los datos
df_games = pd.DataFrame(records)
df_games.to_csv(OUTPUT_FILE, index=False, encoding="utf-8-sig")

print(f"\n Archivo guardado: {OUTPUT_FILE}")
print(f"Total de registros válidos: {len(df_games)}")
print("Columnas disponibles:", ", ".join(df_games.columns))


Descargando lista de juegos de Steam...
Total de juegos en Steam: 273723
Muestra seleccionada: 10000 juegos



Extrayendo datos de juegos: 100%|██████████| 10000/10000 [5:17:53<00:00,  1.91s/it] 


✅ Archivo guardado: steam_games_enriched.csv
Total de registros válidos: 9060
Columnas disponibles: appid, name, type, is_free, price, currency, discount_percent, release_date, metacritic, recommendations, required_age, dlc_count, developers, publishers, categories, genres, player_count, has_economy, asset_count


In [7]:
import pandas as pd

df_games = pd.read_csv("steam_games_final.csv")
df_games


,appid,name,type,is_free,price,currency,discount_percent,release_date,metacritic,recommendations,required_age,dlc_count,developers,publishers,categories,genres,player_count,has_economy,asset_count
0,1084520,雀姬/姫麻雀,game,True,NaN,NaN,NaN,"Aug 5, 2019",NaN,NaN,0,0,Jack Tech,Smile Tech,"Single-player,Multi-player,PvP,Online PvP","Casual,RPG,Strategy,Free To Play,Early Access",69.0,False,NaN
1,830470,MazeBot,game,True,NaN,NaN,NaN,"May 3, 2018",NaN,NaN,0,0,Dynamite Whale,Dynamite Whale,"Single-player,Steam Achievements,Full controll...","Action,Casual,Indie",0.0,False,NaN
2,3680170,Fear To Survive,game,False,NaN,NaN,NaN,Coming soon,NaN,NaN,0,0,Hamuno,Hamuno,"Single-player,Steam Achievements,Family Sharing","Action,Adventure,Casual,Indie,Simulation",NaN,False,NaN
3,3321940,"The Legend of Heroes: Kai no Kiseki -Farewell,...",dlc,False,1425.0,USD,0.0,"Jan 22, 2025",NaN,NaN,0,0,Nihon Falcom,Clouded Leopard Entertainment,"Single-player,Downloadable Content,Family Sharing",RPG,0.0,False,NaN
4,24043,Train Simulator Classic,game,False,2999.0,USD,0.0,"Sep 17, 2015",NaN,18811.0,0,780,Dovetail Games,"Dovetail Games - Trains,Dovetail Games","Single-player,Steam Achievements,Full controll...",Simulation,0.0,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9055,1254950,Hell To Raze,game,True,NaN,NaN,NaN,"Aug 11, 2020",NaN,NaN,0,0,Hellhound Studios,Hellhound Studios,"Single-player,Full controller support","Action,Adventure,Free To Play,Indie",0.0,False,NaN
9056,3454140,Nubs! Playtest,game,False,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,False,NaN
9057,2000180,Office Run,game,True,NaN,NaN,NaN,"May 31, 2022",NaN,NaN,0,1,Adrian Siska,Adrian Siska,"Single-player,Steam Achievements,Full controll...","Action,Casual,Indie,Free To Play",0.0,False,NaN
9058,2340800,Life Changer,game,False,674.0,USD,25.0,"Apr 19, 2023",NaN,NaN,0,0,DCTK,DCTK,"Single-player,Multi-player,PvP,Shared/Split Sc...","Adventure,Casual,Indie,RPG,Simulation,Strategy",0.0,False,NaN


In [ ]:
# Eliminamos filas donde release_date no tenga datos relevantes

df_games = df_games[~df_games['release_date'].str.lower().isin(['coming soon', 'to be announced'])]
df_games['release_date'].str.lower().isin(['coming soon', 'to be announced']).sum()
df_games

np.int64(0)

,appid,name,type,is_free,price,currency,discount_percent,release_date,metacritic,recommendations,required_age,dlc_count,developers,publishers,categories,genres,player_count,has_economy,asset_count
0,1084520,雀姬/姫麻雀,game,True,NaN,NaN,NaN,"Aug 5, 2019",NaN,NaN,0,0,Jack Tech,Smile Tech,"Single-player,Multi-player,PvP,Online PvP","Casual,RPG,Strategy,Free To Play,Early Access",69.0,False,NaN
1,830470,MazeBot,game,True,NaN,NaN,NaN,"May 3, 2018",NaN,NaN,0,0,Dynamite Whale,Dynamite Whale,"Single-player,Steam Achievements,Full controll...","Action,Casual,Indie",0.0,False,NaN
3,3321940,"The Legend of Heroes: Kai no Kiseki -Farewell,...",dlc,False,1425.0,USD,0.0,"Jan 22, 2025",NaN,NaN,0,0,Nihon Falcom,Clouded Leopard Entertainment,"Single-player,Downloadable Content,Family Sharing",RPG,0.0,False,NaN
4,24043,Train Simulator Classic,game,False,2999.0,USD,0.0,"Sep 17, 2015",NaN,18811.0,0,780,Dovetail Games,"Dovetail Games - Trains,Dovetail Games","Single-player,Steam Achievements,Full controll...",Simulation,0.0,False,NaN
6,1235460,Angry Bunny 2: Lost hole,game,False,49.0,USD,51.0,"Mar 13, 2020",NaN,NaN,0,0,VS Resolution,My Way Games,"Single-player,Steam Achievements,Family Sharing","Adventure,Casual,Indie",0.0,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9055,1254950,Hell To Raze,game,True,NaN,NaN,NaN,"Aug 11, 2020",NaN,NaN,0,0,Hellhound Studios,Hellhound Studios,"Single-player,Full controller support","Action,Adventure,Free To Play,Indie",0.0,False,NaN
9056,3454140,Nubs! Playtest,game,False,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,False,NaN
9057,2000180,Office Run,game,True,NaN,NaN,NaN,"May 31, 2022",NaN,NaN,0,1,Adrian Siska,Adrian Siska,"Single-player,Steam Achievements,Full controll...","Action,Casual,Indie,Free To Play",0.0,False,NaN
9058,2340800,Life Changer,game,False,674.0,USD,25.0,"Apr 19, 2023",NaN,NaN,0,0,DCTK,DCTK,"Single-player,Multi-player,PvP,Shared/Split Sc...","Adventure,Casual,Indie,RPG,Simulation,Strategy",0.0,False,NaN


In [23]:
#Ahora lo mismo, pero para todos aquellos que no son "game" en la columna "type"
df_games = df_games[~df_games['type'].str.lower().isin(['dlc','demo','mod','tool','music'])]
df_games['type'].str.lower().isin(['dlc']).sum()
df_games

np.int64(0)

,appid,name,type,is_free,price,currency,discount_percent,release_date,metacritic,recommendations,required_age,dlc_count,developers,publishers,categories,genres,player_count,has_economy,asset_count
0,1084520,雀姬/姫麻雀,game,True,NaN,NaN,NaN,"Aug 5, 2019",NaN,NaN,0,0,Jack Tech,Smile Tech,"Single-player,Multi-player,PvP,Online PvP","Casual,RPG,Strategy,Free To Play,Early Access",69.0,False,NaN
1,830470,MazeBot,game,True,NaN,NaN,NaN,"May 3, 2018",NaN,NaN,0,0,Dynamite Whale,Dynamite Whale,"Single-player,Steam Achievements,Full controll...","Action,Casual,Indie",0.0,False,NaN
4,24043,Train Simulator Classic,game,False,2999.0,USD,0.0,"Sep 17, 2015",NaN,18811.0,0,780,Dovetail Games,"Dovetail Games - Trains,Dovetail Games","Single-player,Steam Achievements,Full controll...",Simulation,0.0,False,NaN
6,1235460,Angry Bunny 2: Lost hole,game,False,49.0,USD,51.0,"Mar 13, 2020",NaN,NaN,0,0,VS Resolution,My Way Games,"Single-player,Steam Achievements,Family Sharing","Adventure,Casual,Indie",0.0,False,NaN
9,2453320,Dobuki's Epic Journey,game,False,199.0,USD,0.0,"Jun 16, 2023",NaN,NaN,0,0,"Jack Le Hamster,Vincent Le Quang",Dobuki Studio,"Single-player,Family Sharing","Adventure,Indie,RPG",0.0,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9054,3098850,Valhalla Path: Survival,game,False,1199.0,USD,0.0,"Jan 6, 2025",NaN,NaN,0,0,Survival Saga Studios,Survival Saga Studios,"Single-player,Family Sharing",RPG,0.0,False,NaN
9055,1254950,Hell To Raze,game,True,NaN,NaN,NaN,"Aug 11, 2020",NaN,NaN,0,0,Hellhound Studios,Hellhound Studios,"Single-player,Full controller support","Action,Adventure,Free To Play,Indie",0.0,False,NaN
9056,3454140,Nubs! Playtest,game,False,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,False,NaN
9057,2000180,Office Run,game,True,NaN,NaN,NaN,"May 31, 2022",NaN,NaN,0,1,Adrian Siska,Adrian Siska,"Single-player,Steam Achievements,Full controll...","Action,Casual,Indie,Free To Play",0.0,False,NaN


In [ ]:
#Creamos dos df, para separar los F2P y los de pago
df_f2p = df_games[df_games['is_free'] == True]
df_con_precio = df_games[df_games['is_free'] == False]
df_con_precio = df_con_precio.dropna(subset=['price']) #Eliminamos precios NaN
df_con_precio['price'] = pd.to_numeric(df_con_precio['price'], errors='coerce') # convertimos a númerico los precios
df_con_precio['price'] = df_con_precio['price'] / 100 # Para generar los precios reales, vienen en céntimos(199 en vez de 1'99)
df_f2p
df_con_precio
print(f"Juegos F2P: {len(df_f2p)}")
print(f"Juegos de pago: {len(df_con_precio)}")



,appid,name,type,is_free,price,currency,discount_percent,release_date,metacritic,recommendations,required_age,dlc_count,developers,publishers,categories,genres,player_count,has_economy,asset_count
0,1084520,雀姬/姫麻雀,game,True,NaN,NaN,NaN,"Aug 5, 2019",NaN,NaN,0,0,Jack Tech,Smile Tech,"Single-player,Multi-player,PvP,Online PvP","Casual,RPG,Strategy,Free To Play,Early Access",69.0,False,NaN
1,830470,MazeBot,game,True,NaN,NaN,NaN,"May 3, 2018",NaN,NaN,0,0,Dynamite Whale,Dynamite Whale,"Single-player,Steam Achievements,Full controll...","Action,Casual,Indie",0.0,False,NaN
33,1069670,EILF,game,True,NaN,NaN,NaN,"Apr 30, 2019",NaN,NaN,0,6,SunshineOvercast,SunshineOvercast,"Single-player,Steam Cloud","Casual,Free To Play,Indie,RPG",0.0,False,NaN
44,2164390,Skader,game,True,NaN,NaN,NaN,"Apr 20, 2023",NaN,NaN,0,0,DreamTeam Mobile,DreamTeam Mobile,"Single-player,Multi-player,PvP,Online PvP,Co-o...","Action,Racing,Sports,Free To Play",0.0,False,NaN
100,3247350,Operation REMODE,game,True,NaN,NaN,NaN,"Jul 28, 2025",NaN,NaN,0,0,Ragoo Rumble,AIV - Accademia Italiana Videogiochi,Single-player,"Action,Adventure,Indie,Free To Play",0.0,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9032,2320130,Farm Empire,game,True,NaN,NaN,NaN,"Mar 23, 2023",NaN,NaN,0,0,Superpog ApS,Playtopia.com,"Single-player,Steam Achievements,In-App Purchases","Casual,Free To Play,Indie",31.0,False,NaN
9045,3899640,Roman Rumble,game,True,NaN,NaN,NaN,"Oct 14, 2025",NaN,NaN,0,0,E-motion Studio,E-motion Studio,"Single-player,Steam Achievements,Stereo Sound","Action,Adventure,Casual,Indie,RPG,Sports,Free ...",0.0,False,NaN
9047,2566030,Animus Toil,game,True,NaN,NaN,NaN,"Dec 22, 2023",NaN,NaN,0,4,Mindset Makers,Mindset Makers,"Single-player,Partial Controller Support","Adventure,Indie",0.0,False,NaN
9055,1254950,Hell To Raze,game,True,NaN,NaN,NaN,"Aug 11, 2020",NaN,NaN,0,0,Hellhound Studios,Hellhound Studios,"Single-player,Full controller support","Action,Adventure,Free To Play,Indie",0.0,False,NaN


,appid,name,type,is_free,price,currency,discount_percent,release_date,metacritic,recommendations,required_age,dlc_count,developers,publishers,categories,genres,player_count,has_economy,asset_count
4,24043,Train Simulator Classic,game,False,29.99,USD,0.0,"Sep 17, 2015",NaN,18811.0,0,780,Dovetail Games,"Dovetail Games - Trains,Dovetail Games","Single-player,Steam Achievements,Full controll...",Simulation,0.0,False,NaN
6,1235460,Angry Bunny 2: Lost hole,game,False,0.49,USD,51.0,"Mar 13, 2020",NaN,NaN,0,0,VS Resolution,My Way Games,"Single-player,Steam Achievements,Family Sharing","Adventure,Casual,Indie",0.0,False,NaN
9,2453320,Dobuki's Epic Journey,game,False,1.99,USD,0.0,"Jun 16, 2023",NaN,NaN,0,0,"Jack Le Hamster,Vincent Le Quang",Dobuki Studio,"Single-player,Family Sharing","Adventure,Indie,RPG",0.0,False,NaN
10,2491460,Mole Royal,game,False,1.99,USD,0.0,"Mar 29, 2024",NaN,NaN,0,0,Hungry Cats Games,Hungry Cats Games,"Single-player,Multi-player,PvP,Online PvP,LAN ...","Action,Casual,Indie",0.0,False,NaN
11,790960,Beer Ranger,game,False,1.99,USD,0.0,"Feb 19, 2018",NaN,NaN,0,0,Mirthood Games,Mirthood Games,"Single-player,Steam Achievements,Partial Contr...","Action,Adventure,Indie",0.0,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9043,2792610,Stickman Killing Zombie,game,False,2.99,USD,0.0,"Apr 12, 2024",NaN,NaN,0,0,Yurii Nikshych,Angels Vs Demons,"Single-player,Steam Achievements,Full controll...","Action,Adventure",0.0,False,NaN
9046,1013140,Outcast - A New Beginning,game,False,39.99,USD,0.0,"Mar 15, 2024",NaN,997.0,0,1,Appeal Studios,THQ Nordic,"Single-player,Steam Achievements,Full controll...",Action,14.0,False,NaN
9049,819560,TurbOT Racing,game,False,0.99,USD,0.0,"Apr 13, 2018",NaN,NaN,0,1,creasso,creasso,"Single-player,Steam Achievements,Full controll...","Action,Indie,Racing",0.0,False,NaN
9054,3098850,Valhalla Path: Survival,game,False,11.99,USD,0.0,"Jan 6, 2025",NaN,NaN,0,0,Survival Saga Studios,Survival Saga Studios,"Single-player,Family Sharing",RPG,0.0,False,NaN


Juegos F2P: 604
Juegos de pago: 3333


In [26]:
# Juegos free-to-play con economía
df_f2p_micro = df_f2p[df_f2p['has_economy'] == True]

# Juegos de pago con economía
df_con_precio_micro = df_con_precio[df_con_precio['has_economy'] == True]
df_f2p_micro
df_con_precio_micro
print(f"Juegos F2P con microtransacciones : {len(df_f2p_micro)}")
print(f"Juegos de pago con microtransacciones: {len(df_con_precio_micro)}")



,appid,name,type,is_free,price,currency,discount_percent,release_date,metacritic,recommendations,required_age,dlc_count,developers,publishers,categories,genres,player_count,has_economy,asset_count


,appid,name,type,is_free,price,currency,discount_percent,release_date,metacritic,recommendations,required_age,dlc_count,developers,publishers,categories,genres,player_count,has_economy,asset_count


Juegos F2P con microtransacciones : 0
Juegos de pago con microtransacciones: 0
